In [1]:
# Polytope Definition

In [2]:
#| default_exp polytope

In [3]:
#| export
import torch
import numpy as np

from typing import  Union
import gc

from fastcore.basics import patch

In [11]:
#| export

class Polytope:
    
    def __init__(self, 
                A_in:Union[torch.Tensor, np.ndarray], 
                b_in:Union[torch.Tensor, np.ndarray], 
                dtype=torch.float16,
                copy:bool=False,
                requires_grad:bool=False
                 ) -> None:
        
        self._check_intra_constraint_dimension_(A_in,b_in,'Inequality')
        
        self.copy = copy
        if requires_grad:
            print('The tensors will accumalate the gradient of the operations')
        self.requires_grad = requires_grad
        
        self.dtype = dtype
        self.mI = A_in.shape[0]
        self.n = A_in.shape[1]        
        self.A_in = self._process_tensor_or_array(A_in,'A_in')
        self.b_in = self._process_tensor_or_array(b_in,'b_in')
        gc.collect()
    
    
    def _check_intra_constraint_dimensions_(self, 
                        A:Union[torch.Tensor, np.ndarray], 
                        b:Union[torch.Tensor, np.ndarray], 
                        constraint:str=None):
    
        assert(A.shape[0] == b.shape[0]), f"{constraint} has dimension mismatch: A has shape {A.shape} and b {b.shape}."
    
    
    def _process_tensor_or_array_(self, 
                                 A:Union[torch.Tensor, np.ndarray], 
                                 restriction:str=None):
        gc.collect()
        dtype = self.dtype
        if isinstance(A, torch.Tensor): # Torch Tensor
            # A is already a PyTorch tensor
            if self.copy:
                tensor_A = A.clone()
            else:
                tensor_A = A
            if dtype is not None and A.dtype != dtype:
                tensor_A = tensor_A.to(dtype)
            return tensor_A.requires_grad_(self.requires_grad)
        elif isinstance(A, np.ndarray): # Numpy array
            # A is a NumPy array, convert it to a PyTorch tensor
            if self.copy:
                tensor_A = torch.tensor(A, dtype=dtype)
            else:
                tensor_A = torch.from_numpy(A, dtype=dtype)
            return tensor_A.requires_grad_(self.requires_grad)
        else:
            raise ValueError(f"Input {restriction} must be a NumPy array or PyTorch tensor")

    

In [12]:
#| export
@patch
def __str__(self:Polytope):
    string = f'Numeric Precision (dtype) {self.dtype}'
    string = string + f'A_in: {self.A_in.shape} \n'
    string = string + f'b_in: {self.b_in.shape}'
    return string

In [8]:
#| export
class NFDPolytope(Polytope):
    
    def __init__(self, 
                A_in:Union[torch.Tensor, np.ndarray], 
                b_in:Union[torch.Tensor, np.ndarray], 
                A_eq:Union[torch.Tensor, np.ndarray], 
                b_eq:Union[torch.Tensor, np.ndarray], 
                dtype=torch.float32 # Default dtype for Non-Fully-Dimensioonal Polytopes is 32 bits
                 ) -> None:
        
        self._check_intra_constraint_dimension_(A_in,b_in,'Inequality')
        self._check_intra_constraint_dimension_(A_eq,b_eq,'Equality')
        self._check_inter_constraint_dimensions_(A_in, A_eq)

        super.__init__(self, A_in, b_in, dtype)
        self.mE = A_eq.shape[0]
        self.A_eq = self._process_tensor_or_array(A_eq,'A_eq')
        self.b_eq = self._process_tensor_or_array(b_eq,'b_eq')
        gc.collect()
        
        
    def _check_inter_constraint_dimensions_(self,A_in,A_eq):
        
        assert(A_in.shape[0] == A_eq.shape[0]), f"Constraints have dimension mismatch: A_in has shape {A_in.shape} and  A_eq {A_eq.shape}."
        
        

In [9]:
#| export
@patch
def __str__(self:NFDPolytope):
    string = super.__str__()
    string = string + f'A_eq: {self.A_eq.shape} \n b_eq: {self.b_eq.shape}'
    return string